# Experiments with backtesting.py
This notebook is to explore the `backtesting.py` to evaluate strategy:
1. Based on conventional TA (talib etc.)
2. See how can we introduce ML model as TA

Install ta-lib on Mac with brew first `brew install ta-lib` and then `pip install TA-Lib`

In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import yfinance as yf
import talib 
from utils import get_stock_data
import time
import warnings

/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### 1. Standard Examples
First, use standard example; where goal is just to see how we can use the library. In the following, we will use SMA as an indicator which is already available in `backtesting.py`. We will also use stop loss and take profit to limit the exposure and to ensure risk-reward ratio.

In [2]:
class SmaCrossWithSLTP(Strategy):
    short_window = 10  # Fast SMA
    long_window = 20   # Slow SMA
    stop_loss_pct = 0.02   # 2% Stop Loss
    take_profit_pct = 0.04  # 4% Take Profit
    
    # All the necessary indicator needs to define in the init method
    def init(self):
        self.sma_short = self.I(SMA, self.data.Close, self.short_window)
        self.sma_long = self.I(SMA, self.data.Close, self.long_window)
    
    # It defines the logic of the strategy
    def next(self):
        price = self.data.Close[-1]  # Current price
        if crossover(self.sma_short, self.sma_long):  # Buy condition
            stop_loss = price * (1 - self.stop_loss_pct)
            take_profit = price * (1 + self.take_profit_pct)
            self.buy(sl=stop_loss, tp=take_profit)  # Order with SL and TP
        
        elif crossover(self.sma_long, self.sma_short):  # Sell condition
            self.position.close()  # Close existing position

# Load sample data which comes with the library to test
df = GOOG

# Run Backtest
bt = Backtest(df, SmaCrossWithSLTP, cash=10000, commission=0.002)
stats = bt.run()
print(stats)
# bt.plot()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     9.82309
Equity Final [$]                  11532.01791
Equity Peak [$]                   12667.87271
Commissions [$]                    2041.36509
Return [%]                           15.32018
Buy & Hold Return [%]               607.37036
Return (Ann.) [%]                     1.68634
Volatility (Ann.) [%]                 6.32304
CAGR [%]                              1.15945
Sharpe Ratio                           0.2667
Sortino Ratio                         0.40031
Calmar Ratio                          0.14875
Alpha [%]                            -4.51391
Beta                                  0.03266
Max. Drawdown [%]                   -11.33648
Avg. Drawdown [%]                    -3.53535
Max. Drawdown Duration      742 days 00:00:00
Avg. Drawdown Duration      172 days 00:00:00
# Trades                          

## 2a. Integrating other indicators from TA-LIB
Now, moving to `ta-lib` for more indicators. In the following, we will use RSI along with SL and TP.

In [6]:
class RsiSmaStrategy(Strategy):
    rsi_period = 14    # RSI period
    rsi_buy = 30       # RSI oversold threshold
    rsi_sell = 70      # RSI overbought threshold
    stop_loss_pct = 0.02  # 2% stop-loss
    take_profit_pct = 0.05  # 5% take-profit

    def init(self):    
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Entry Condition: RSI below 30 & SMA crossover
        if self.rsi[-1] < self.rsi_buy :
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  # Stop-Loss 2% below entry
            tp = price * (1 + self.take_profit_pct)  # Take-Profit 5% above entry
            self.buy(sl=sl, tp=tp)  # Place order with SL & TP

        # Exit Condition: RSI above 70 OR SMA crossover in opposite direction
        elif self.rsi[-1] > self.rsi_sell:
            self.position.close()

df = get_stock_data("MSFT", interval="1d", start="2024-02-01", end="2025-03-31")

# Run Backtest
bt = Backtest(df, RsiSmaStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
# Save HTML file for plots for in-browser visualization
# bt.plot(filename="rsi_strategy.html")

Backtest.run:   0%|          | 0/275 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=129: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=130: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                     

Start                     2024-02-01 00:00:00
End                       2025-03-28 00:00:00
Duration                    421 days 00:00:00
Exposure Time [%]                     2.41379
Equity Final [$]                  10278.12869
Equity Peak [$]                   10278.12869
Commissions [$]                      77.58101
Return [%]                            2.78129
Buy & Hold Return [%]                -7.27754
Return (Ann.) [%]                     2.41248
Volatility (Ann.) [%]                 2.13113
CAGR [%]                              1.65563
Sharpe Ratio                          1.13202
Sortino Ratio                         4.63038
Calmar Ratio                          2.53299
Alpha [%]                             2.86155
Beta                                  0.01103
Max. Drawdown [%]                    -0.95243
Avg. Drawdown [%]                    -0.95243
Max. Drawdown Duration        6 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                          

## 2b. Using pandas_ta  
`pandas_ta` is a simpler wrapper for dataframe. Some of the TA are build upon above-used `ta-lib` so install both.

In [7]:
class RsiSmaStrategy(Strategy):
    rsi_period = 14
    rsi_buy = 30
    rsi_sell = 70
    stop_loss_pct = 0.02
    take_profit_pct = 0.05

    def init(self):
        # Compute RSI using pandas_ta
        rsi_series = self.data.df.ta.rsi(length=self.rsi_period)
        self.rsi = self.I(lambda: rsi_series)

    def next(self):
        if self.rsi[-1] < self.rsi_buy:
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)
            tp = price * (1 + self.take_profit_pct)
            self.buy(sl=sl, tp=tp)

        elif self.rsi[-1] > self.rsi_sell:
            self.position.close()

# Assuming get_stock_data returns a DataFrame with OHLCV
df = get_stock_data("MSFT", interval="1d", start="2024-02-01", end="2025-03-31")

# Run Backtest
bt = Backtest(df, RsiSmaStrategy, cash=10000, commission=0.002)
stats = bt.run()

print(stats)
# bt.plot(filename="rsi_strategy.html")


Backtest.run:   0%|          | 0/275 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=129: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=130: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                     

Start                     2024-02-01 00:00:00
End                       2025-03-28 00:00:00
Duration                    421 days 00:00:00
Exposure Time [%]                     2.41379
Equity Final [$]                  10278.12869
Equity Peak [$]                   10278.12869
Commissions [$]                      77.58101
Return [%]                            2.78129
Buy & Hold Return [%]                -7.27754
Return (Ann.) [%]                     2.41248
Volatility (Ann.) [%]                 2.13113
CAGR [%]                              1.65563
Sharpe Ratio                          1.13202
Sortino Ratio                         4.63038
Calmar Ratio                          2.53299
Alpha [%]                             2.86155
Beta                                  0.01103
Max. Drawdown [%]                    -0.95243
Avg. Drawdown [%]                    -0.95243
Max. Drawdown Duration        6 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                          

#### More TA based strategigies (Bollinger band)

In [8]:
class RsiBollingerStrategy(Strategy):
    rsi_period = 14
    rsi_buy = 30
    rsi_sell = 70
    bb_period = 20
    bb_std = 2
    stop_loss_pct = 0.02
    take_profit_pct = 0.05

    def init(self):
        df = self.data.df

        # Indicators from pandas_ta
        self.rsi = self.I(lambda: df.ta.rsi(length=self.rsi_period))
        bb = df.ta.bbands(length=self.bb_period, std=self.bb_std)

        # Use lower & upper bands from BB
        self.bb_lower = self.I(lambda: bb['BBL_20_2.0'])
        self.bb_upper = self.I(lambda: bb['BBU_20_2.0'])

    def next(self):
        price = self.data.Close[-1]

        # Buy condition: RSI < 30 and price below lower BB
        if self.rsi[-1] < self.rsi_buy and price < self.bb_lower[-1]:
            sl = price * (1 - self.stop_loss_pct)
            tp = price * (1 + self.take_profit_pct)
            self.buy(sl=sl, tp=tp)

        # Sell condition: RSI > 70 or price > upper BB
        elif self.position:
            if self.rsi[-1] > self.rsi_sell or price > self.bb_upper[-1]:
                self.position.close()

# Fetch data
df = get_stock_data("MSFT", interval="1d", start="2024-02-01", end="2025-03-31")

# Run backtest
bt = Backtest(df, RsiBollingerStrategy, cash=10000, commission=0.002)
stats = bt.run()

print(stats)
# bt.plot(filename="rsi_bb_strategy.html")


Start                     2024-02-01 00:00:00
End                       2025-03-28 00:00:00
Duration                    421 days 00:00:00
Exposure Time [%]                     2.06897
Equity Final [$]                  10330.02134
Equity Peak [$]                   10330.02134
Commissions [$]                      40.50959
Return [%]                            3.30021
Buy & Hold Return [%]                -7.72363
Return (Ann.) [%]                     2.86165
Volatility (Ann.) [%]                 2.18469
CAGR [%]                              1.96254
Sharpe Ratio                          1.30987
Sortino Ratio                         7.34631
Calmar Ratio                           4.8579
Alpha [%]                             3.36926
Beta                                  0.00894
Max. Drawdown [%]                    -0.58907
Avg. Drawdown [%]                    -0.58907
Max. Drawdown Duration        3 days 00:00:00
Avg. Drawdown Duration        3 days 00:00:00
# Trades                          

Giving control to LLM for "best" startegy (generated by LLM)

In [15]:
class RsiWmaBollingerStrategy(Strategy):
    rsi_period = 10
    rsi_buy = 28
    rsi_sell = 60
    bb_period = 15
    bb_std = 1.5
    stop_loss_pct = 0.02
    take_profit_pct = 0.05

    def init(self):
        close = self.data.Close

        # Use TA-Lib for all indicators (multiprocessing safe)
        self.rsi = talib.RSI(close, timeperiod=self.rsi_period)
        self.wma = talib.WMA(close, timeperiod=self.bb_period)
        self.std = pd.Series(close).rolling(self.bb_period).std().values

        self.bb_upper = self.wma + self.bb_std * self.std
        self.bb_lower = self.wma - self.bb_std * self.std

    def next(self):
        i = len(self.data.Close) - 1
        price = self.data.Close[i]

        # Handle edge cases
        if np.isnan(self.rsi[i]) or np.isnan(self.bb_lower[i]) or np.isnan(self.bb_upper[i]):
            return

        if self.rsi[i] < self.rsi_buy and price < self.bb_lower[i]:
            sl = price * (1 - self.stop_loss_pct)
            tp = price * (1 + self.take_profit_pct)
            self.buy(sl=sl, tp=tp)

        elif self.position:
            if self.rsi[i] > self.rsi_sell or price > self.bb_upper[i]:
                self.position.close()



# Fetch data
df = get_stock_data("MSFT", interval="1d", start="2024-02-01", end="2025-03-31")

# Run backtest
bt = Backtest(df, RsiWmaBollingerStrategy, cash=10000, commission=0.002)
stats = bt.run()

print(stats)
#bt.plot(filename="RsiWmaBollingerStrategy.html")


Start                     2024-02-01 00:00:00
End                       2025-03-28 00:00:00
Duration                    421 days 00:00:00
Exposure Time [%]                     5.51724
Equity Final [$]                   9817.83811
Equity Peak [$]                   10202.94688
Commissions [$]                     114.40817
Return [%]                           -1.82162
Buy & Hold Return [%]                 -5.2955
Return (Ann.) [%]                    -1.58483
Volatility (Ann.) [%]                 5.53378
CAGR [%]                              -1.0944
Sharpe Ratio                         -0.28639
Sortino Ratio                        -0.33089
Calmar Ratio                         -0.21765
Alpha [%]                            -1.54961
Beta                                  0.05137
Max. Drawdown [%]                    -7.28145
Avg. Drawdown [%]                    -7.28145
Max. Drawdown Duration      338 days 00:00:00
Avg. Drawdown Duration      338 days 00:00:00
# Trades                          

One more trial

In [17]:
class MacdVwapStrategy(Strategy):
    macd_fast = 15
    macd_slow = 50
    macd_signal = 9
    stop_loss_pct = 0.015  # 1.5%
    take_profit_pct = 0.03  # 3%

    def init(self):
        close = self.data.Close
        high = self.data.High
        low = self.data.Low
        volume = self.data.Volume

        # MACD indicator
        self.macd, self.macd_signal_line, _ = talib.MACD(close,
                                                         fastperiod=self.macd_fast,
                                                         slowperiod=self.macd_slow,
                                                         signalperiod=self.macd_signal)

        # VWAP: cumulative typical price * volume / cumulative volume
        typical_price = (high + low + close) / 3
        cumulative_tpv = (typical_price * volume).cumsum()
        cumulative_volume = volume.cumsum()
        self.vwap = (cumulative_tpv / cumulative_volume)

    def next(self):
        i = len(self.data.Close) - 1
        price = self.data.Close[i]

        if i < 1 or np.isnan(self.macd[i]) or np.isnan(self.macd_signal_line[i]) or np.isnan(self.vwap[i]):
            return

        macd_cross_up = self.macd[i - 1] < self.macd_signal_line[i - 1] and self.macd[i] > self.macd_signal_line[i]
        macd_cross_down = self.macd[i - 1] > self.macd_signal_line[i - 1] and self.macd[i] < self.macd_signal_line[i]

        above_vwap = price > self.vwap[i]
        below_vwap = price < self.vwap[i]

        if not self.position:
            # Long position
            if macd_cross_up and above_vwap:
                sl = price * (1 - self.stop_loss_pct)
                tp = price * (1 + self.take_profit_pct)
                self.buy(sl=sl, tp=tp)

            # Short position
            elif macd_cross_down and below_vwap:
                sl = price * (1 + self.stop_loss_pct)
                tp = price * (1 - self.take_profit_pct)
                self.sell(sl=sl, tp=tp)

        # Optional exit logic (early exit on MACD flip)
        elif self.position.is_long and macd_cross_down:
            self.position.close()

        elif self.position.is_short and macd_cross_up:
            self.position.close()

# Fetch data
df = get_stock_data("MSFT", interval="1d", start="2024-02-01", end="2025-03-31")

# Run backtest
bt = Backtest(df, MacdVwapStrategy, cash=10000, commission=0.002)
stats = bt.run()

print(stats)
#bt.plot(filename="MacdVwapStrategy.html")


Start                     2024-02-01 00:00:00
End                       2025-03-28 00:00:00
Duration                    421 days 00:00:00
Exposure Time [%]                    22.75862
Equity Final [$]                   8632.22792
Equity Peak [$]                   10725.52251
Commissions [$]                     578.24784
Return [%]                          -13.67772
Buy & Hold Return [%]                 -5.2955
Return (Ann.) [%]                   -11.99791
Volatility (Ann.) [%]                 8.73633
CAGR [%]                             -8.42756
Sharpe Ratio                         -1.37333
Sortino Ratio                        -1.41553
Calmar Ratio                          -0.5444
Alpha [%]                            -13.3477
Beta                                  0.06232
Max. Drawdown [%]                   -22.03875
Avg. Drawdown [%]                    -6.59797
Max. Drawdown Duration      172 days 00:00:00
Avg. Drawdown Duration       63 days 00:00:00
# Trades                          

At this point, we have few strategies which works good for some and not so good for others. So let's filter/screen positive stock for us

In [18]:
def run_multi_stock_backtest(tickers,  period="60d"):
    results = []

    for ticker in tickers:
        time.sleep(2.3)
        print(f"Running backtest for {ticker}...")
        try:
            df = get_stock_data(ticker, interval="15m", period="60d")

            if df.empty:
                continue

            bt = Backtest(df, MacdVwapStrategy, cash=10_000, commission=0.002)     
            stats = bt.run()

            results.append({
                "Ticker": ticker,
                "Return [%]": stats['Return [%]'],
                "Win Rate [%]": stats['Win Rate [%]'],
                "Sharpe Ratio": stats.get('Sharpe Ratio', None),
                "Trades": stats['# Trades'],
                "Avg Trade Duration": stats['Avg. Trade Duration'],
                "Exposure Time [%]": stats['Exposure Time [%]'],
            })

        except Exception as e:
            print(f"Error with {ticker}: {e}")

    df_results = pd.DataFrame(results)
    #df_results.sort_values(by="Return [%]", ascending=False, inplace=True)
    return df_results

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NFLX", "META", "NVDA", "AMD", "INTC"]
results = run_multi_stock_backtest(tickers,  period="60d")
print(results)


Running backtest for AAPL...


Running backtest for MSFT...


Running backtest for GOOGL...


Running backtest for AMZN...


Running backtest for TSLA...


...

Running backtest for META...


Running backtest for NVDA...


Running backtest for AMD...


Running backtest for INTC...


  Ticker  Return [%]  Win Rate [%]  Sharpe Ratio  Trades Avg Trade Duration  \
0   AAPL  -17.800237     33.333333     -5.248013      48    0 days 10:30:00   
1   MSFT   -0.881895     45.000000     -0.236849      40    0 days 18:00:00   
2  GOOGL  -25.199270     35.555556     -7.935699      45    0 days 12:15:00   
3   AMZN    3.052927     44.444444      0.499754      45    0 days 09:50:00   
4   TSLA    7.486382     45.652174      0.784061      46    0 days 10:46:00   
5   NFLX  -14.215344     24.444444     -4.139055      45    0 days 12:53:00   
6   META    3.722975     52.173913      0.564890      46    0 days 10:42:00   
7   NVDA  -19.848279     35.555556     -5.223692      45    0 days 10:32:00   
8    AMD  -17.032302     33.333333     -4.065435      45    0 days 13:51:00   
9   INTC  -28.526377     34.693878     -7.572772      49    0 days 09:22:00   

   Exposure Time [%]  
0          35.897436  
1          37.564103  
2          33.782051  
3          36.730769  
4          21.1

## 3. Integrating a black-box indicator (e.g., ML model)
Now we will add a black-box function which can be an ML model which can provide BUY or SELL signal. We will pass the dataframe containing the features and function can use this to predict the position. This then can be used in `next` function.

In [19]:
def dummy_ml_model(df):
    """
    Simulated ML model that predicts BUY signals. We don't consider SELL (short positions).
    :param df: DataFrame of last `N` rows (OHLCV features).
    :return: 1 if BUY signal, else 0.
    """
    # Simulated logic: Buy if the last row closes lower than the first row in the window
    if df.iloc[-1]['Close'] < df.iloc[0]['Close']:  
        return 1  # BUY Signal
    return 0  # No action

class MLTradingStrategy(Strategy):
    feature_window = 5 # window size for features 
    stop_loss_pct = 0.02 
    take_profit_pct = 0.05 

    def init(self):
        pass

    def next(self):
        N = self.feature_window  
        # Ensure enough data to generate features
        if len(self.data.Close) < N:
            return 

        # Extract last `N` rows into a DataFrame
        df = pd.DataFrame({
            "Open": self.data.Open[-N:],
            "High": self.data.High[-N:],
            "Low": self.data.Low[-N:],
            "Close": self.data.Close[-N:],
            "Volume": self.data.Volume[-N:],
        })

        # Pass last `N` rows to ML model
        signal = dummy_ml_model(df)  

        # ML suggests BUY signal
        if signal == 1:  
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  
            tp = price * (1 + self.take_profit_pct) 
            self.buy(sl=sl, tp=tp)  

df = get_stock_data("MSFT", interval="1d", start="2023-02-01", end="2025-03-20")

# Run Backtest
bt = Backtest(df, MLTradingStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
#bt.plot(filename="ml__strategy.html")

Backtest.run:   0%|          | 0/533 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=15: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=16: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=17: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=18: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=20: Bro

Start                     2023-02-01 00:00:00
End                       2025-03-19 00:00:00
Duration                    777 days 00:00:00
Exposure Time [%]                    67.79026
Equity Final [$]                   7589.69493
Equity Peak [$]                    11215.1547
Commissions [$]                    2703.35983
Return [%]                          -24.10305
Buy & Hold Return [%]                56.27101
Return (Ann.) [%]                   -12.20361
Volatility (Ann.) [%]                16.00679
CAGR [%]                             -8.55629
Sharpe Ratio                          -0.7624
Sortino Ratio                        -0.86536
Calmar Ratio                         -0.36042
Alpha [%]                           -54.51222
Beta                                  0.54041
Max. Drawdown [%]                   -33.85963
Avg. Drawdown [%]                    -8.31362
Max. Drawdown Duration      342 days 00:00:00
Avg. Drawdown Duration       82 days 00:00:00
# Trades                          

## 4. Integrating ESN model from `1_esn_le2e.ipynb`
Here, our function will look for some threshold above which we can have BUY. Remember we should not have overlap between training and current prediction period. Otherwise, it's overfitting.

In [20]:
from utils import StockPricePredictor

predictor = StockPricePredictor(model_path="stock_esn_model_7.pkl", 
                                scaler_path="scaler_esn_model_7.pkl")

def if_buy_signal_with_esn(df, threshold=0.02):
    """
    Determines if a BUY signal should be generated based on the ESN model's prediction.
    
    Args:
        df (DataFrame): The dataframe containing the latest stock data.
        threshold (float): The minimum percentage increase required to trigger a BUY signal.
    
    Returns:
        int: 1 if BUY signal is triggered, otherwise 0.
    """
    predicted_close = predictor.predict(df)
    last_close = df.iloc[-1]['Close']
    
    # Check if predicted close is at least (1 + threshold)% higher than last close
    if predicted_close >= last_close * (1 + threshold):
        return 1  # BUY Signal
    
    return 0  # No action


class MLTradingStrategy(Strategy):
    feature_window = 7 # window size for features 
    stop_loss_pct = 0.025 
    take_profit_pct = 0.05 

    def init(self):
        pass

    def next(self):
        N = self.feature_window  
        # Ensure enough data to generate features
        if len(self.data.Close) < N:
            return 

        # Extract last `N` rows into a DataFrame
        df = pd.DataFrame({
            "Open": self.data.Open[-N:],
            "High": self.data.High[-N:],
            "Low": self.data.Low[-N:],
            "Close": self.data.Close[-N:],
            #"Volume": self.data.Volume[-N:],
        })

        # Pass last `N` rows to ML model
        signal = if_buy_signal_with_esn(df)  

        # ML suggests BUY signal
        if signal == 1:  
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  
            tp = price * (1 + self.take_profit_pct) 
            self.buy(sl=sl, tp=tp)  

df = get_stock_data("MSFT", interval="1d", start="2024-02-01", end="2025-03-31")

# Run Backtest
bt = Backtest(df, MLTradingStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
# bt.plot(filename="esn_strategy.html")

Using Numpy backend.


Start                     2024-02-01 00:00:00
End                       2025-03-28 00:00:00
Duration                    421 days 00:00:00
Exposure Time [%]                     4.13793
Equity Final [$]                   8681.88927
Equity Peak [$]                       10000.0
Commissions [$]                     147.36519
Return [%]                          -13.18111
Buy & Hold Return [%]                 -5.2955
Return (Ann.) [%]                   -11.55813
Volatility (Ann.) [%]                 4.42894
CAGR [%]                             -8.11259
Sharpe Ratio                         -2.60969
Sortino Ratio                        -2.29486
Calmar Ratio                         -0.87687
Alpha [%]                           -12.94818
Beta                                  0.04399
Max. Drawdown [%]                   -13.18111
Avg. Drawdown [%]                   -13.18111
Max. Drawdown Duration      259 days 00:00:00
Avg. Drawdown Duration      259 days 00:00:00
# Trades                          

# Predicting for today

In [21]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')
df = get_stock_data(ticker="MSFT", interval="1d", start="2025-03-20", end=today)
df = df.drop(columns=['Volume'])
df.head(10)

Price,Open,High,Low,Close
Date,,,,
2025-03-20,385.739990,391.790009,383.279999,386.839996
2025-03-21,383.220001,391.739990,382.799988,391.260010
2025-03-24,395.399994,395.399994,389.809998,393.079987
2025-03-25,393.920013,396.359985,392.640015,395.160004
2025-03-26,395.000000,395.309998,388.570007,389.970001
2025-03-27,390.130005,392.239990,387.399994,390.579987
2025-03-28,388.079987,389.130005,376.929993,378.799988
2025-03-31,372.540009,377.070007,367.239990,375.390015
2025-04-01,374.649994,382.850006,373.230011,382.190002


In [22]:
signal = if_buy_signal_with_esn(df.tail(7)) 
print("signal", signal)
predicted_close = predictor.predict(df.tail(7))
print("predicted_close", predicted_close) 

signal 0
predicted_close 411.09312788357875


END OF NOTEBOOK